# Capítulo 07 — Instruction Tuning: Criando um Assistente

🎯 **Objetivos:** Transformar o modelo completador em um assistente útil usando **SFT (Supervised Fine-Tuning)**.

![SFT](./infograficos/04-pipeline-sft.png)

In [ ]:
# ============================================================
# Setup do repositório no Colab
# ============================================================
import os, sys
REPO_NAME = "fazendo-um-llm-do-zero"
if 'google.colab' in str(get_ipython()):
    if not os.path.exists(REPO_NAME):
        get_ipython().system(f"git clone https://github.com/vongrossi/{REPO_NAME}.git")
    if os.path.exists(REPO_NAME) and os.getcwd().split('/')[-1] != REPO_NAME:
        os.chdir(REPO_NAME)
if os.getcwd() not in sys.path: sys.path.append(os.getcwd())
print("📂 Diretório atual:", os.getcwd())

In [ ]:
import os, sys, torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from lib.gptmini import GPTConfig, GPTMini

device = "cuda" if torch.cuda.is_available() else "cpu"

# 📂 Carregamento do Checkpoint do Cap 05
if not os.path.exists("gpt_checkpoint.pt"):
    from google.colab import files
    print("📤 Por favor, suba o 'gpt_checkpoint.pt' gerado no Capítulo 05:")
    uploaded = files.upload()

ckpt = torch.load("gpt_checkpoint.pt", map_location=device, weights_only=False)
stoi, itos = ckpt['stoi'], ckpt['itos']
config = ckpt['config']
context_size = config.context_size

# Encoder que ignora caracteres desconhecidos com aviso
def encode(s):
    res = []
    for c in s.lower():
        if c in stoi: res.append(stoi[c])
    return res

decode = lambda l: ''.join([itos[i] for i in l])

print(f"🧠 Modelo pré-treinado carregado!")
print(f"📏 Contexto: {context_size} | Vocabulário: {len(stoi)} caracteres")
if '#' not in stoi: print("⚠️ AVISO: Seu checkpoint não possui o caractere '#'. Re-execute o Capítulo 05 com o novo dataset.")

## 1. Dataset de Instruções

Criamos pares de Pergunta e Resposta para o alinhamento.

In [ ]:
instructions = [
    {"q": "o que o gato fez?", "a": "o gato subiu no telhado e pulou o muro."},
    {"q": "onde o cachorro dormiu?", "a": "o cachorro dormiu no sofa e no tapete."},
    {"q": "defina inteligencia artificial", "a": "inteligencia artificial e o estudo de algoritmos."},
    {"q": "o que e machine learning?", "a": "machine learning permite que sistemas aprendam padroes."}
]

def build_sft_dataset(data, context_size):
    X, Y, masks = [], [], []
    for item in data:
        cmd = f"### comando:\n{item['q']}\n### resposta:\n"
        full = cmd + item['a']
        ids = encode(full)
        cmd_len = len(encode(cmd))
        for i in range(len(ids) - context_size):
            X.append(ids[i : i+context_size])
            Y.append(ids[i+1 : i+context_size+1])
            # Máscara: 0 no comando, 1 na resposta
            m = []
            for j in range(i, i + context_size):
                if j < cmd_len: m.append(0)
                else: m.append(1)
            masks.append(m)
    return torch.tensor(X).to(device), torch.tensor(Y).to(device), torch.tensor(masks).to(device)

X, Y, M = build_sft_dataset(instructions, context_size)
print(f"📦 Amostras de Alinhamento: {len(X)}")

## 2. Treinamento com Máscara de Loss

Otimizamos apenas a geração da resposta.

![Masking](./infograficos/03-mascaramento-loss-resposta.png)

In [ ]:
model = GPTMini(config).to(device)
model.load_state_dict(ckpt['state_dict'])
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)

loss_history = []
model.train()
print("🔨 Alinhando o assistente...")
for step in range(801):
    idx = torch.randint(len(X), (8,))
    logits, _ = model(X[idx])
    B, T, V = logits.shape
    loss = F.cross_entropy(logits.view(-1, V), Y[idx].view(-1), reduction='none')
    loss = (loss * M[idx].view(-1)).mean()
    optimizer.zero_grad(set_to_none=True); loss.backward(); optimizer.step()
    loss_history.append(loss.item())
    if step % 200 == 0: print(f"Step {step} | Loss {loss.item():.4f}")

plt.plot(loss_history, color='#34A853')
plt.title("Curva de Alinhamento (SFT)")
plt.show()

## 3. Teste do Assistente Alinhado

O modelo agora responde apenas o que foi solicitado.

In [ ]:
@torch.no_grad()
def ask(model, question):
    model.eval()
    prompt_text = f"### comando:\n{question.lower()}\n### resposta:\n"
    
    # Debug: mostrar o que o modelo está realmente lendo
    unknowns = [c for c in prompt_text if c not in stoi]
    if unknowns: print(f"⚠️ Ignorando: {set(unknowns)}")
    
    tokens = encode(prompt_text)
    idx = torch.tensor(tokens).unsqueeze(0).to(device)
    prompt_len = len(tokens)
    
    for _ in range(60):
        idx_cond = idx[:, -context_size:]
        logits, _ = model(idx_cond)
        next_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        idx = torch.cat([idx, next_id], dim=1)
        if itos[next_id.item()] == ".": break
        
    # Retornamos apenas a parte gerada (Resposta)
    return decode(idx[0][prompt_len:].tolist())

print("🤖 TESTE DE INTERAÇÃO:")
print("-" * 30)
q1 = "o que o gato fez?"
print(f"Pergunta: {q1}\nResposta: {ask(model, q1)}")

print("\n" + "-" * 30)
q2 = "o que e machine learning?"
print(f"Pergunta: {q2}\nResposta: {ask(model, q2)}")

## 🏁 Conclusão da Jornada

Você completou a série! 

Transformou um modelo estatístico em um assistente capaz de seguir intenções humanas. Este é o fundamento do alinhamento de IA.

![Avaliação](./infograficos/05-avaliacao-respostas.png)